# Neural Networks

## Define the network

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [5]:
net = Net()

net

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)

The learnable parameters of a model are returned by net.parameters()

In [6]:
net.parameters()

<generator object Module.parameters at 0x7fa1d49ef728>

In [7]:
params = list(net.parameters())

len(params)

10

In [8]:
params[0].size()

torch.Size([6, 1, 5, 5])

nn.hogeのレイヤーとかは勝手にparameterとか設定してくれてるっぽい

In [9]:
input = Variable(torch.randn(1,1,32,32))
out = net(input)

out

Variable containing:
1.00000e-02 *
 -1.6245  7.1931 -2.2744  9.2811 -0.9909  4.8597 -4.8555 -9.2346  0.3680 -2.8691
[torch.FloatTensor of size 1x10]

勾配のゼロ初期化を行う

In [10]:
net.zero_grad()

In [12]:
out.backward(torch.randn(1,10))

## Loss Function

In [13]:
output = net(input)
target = Variable(torch.arange(1, 11))

criterion = nn.MSELoss()

In [14]:
loss = criterion(output, target)

loss

Variable containing:
 38.6348
[torch.FloatTensor of size 1]

.creatorには計算グラフが入っている。

計算グラフは.previous_functionsを遡るとその前にやった関数が見れる

In [15]:
loss.creator

In [17]:
loss.creator.previous_functions[0][0]

In [18]:
loss.creator.previous_functions[0][0].previous_functions[0][0]

### Backprop

In [19]:
net.zero_grad()

In [20]:
print("conv1.bias.grad before backward")
net.conv1.bias.grad

conv1.bias.grad before backward


Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

In [21]:
loss.backward()

In [22]:
print('conv1.bias.grad after backward')
net.conv1.bias.grad

conv1.bias.grad after backward


Variable containing:
1.00000e-02 *
 -4.0049
 -7.9969
  0.6708
 -7.3854
 -3.7967
 -3.1010
[torch.FloatTensor of size 6]

## Update the weights

普通のSGDを自分で書くとこんな感じになる

In [23]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

でも実はtorch.optimに確率的勾配降下法は色々実装されているので、普通に使うならこれを使おう

In [24]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update